In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [2]:
import torch, torchvision, torchaudio
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import math

In [5]:
att=torch.tril(torch.ones(3,3).view(1,1,3,3))
print(att)

tensor([[[[1., 0., 0.],
          [1., 1., 0.],
          [1., 1., 1.]]]])


In [51]:
class Positional_encoding(nn.Module):
  def __init__(self):
    super(Positional_encoding, self).__init__()
    return

  def forward(self,x):
    B,T,C=x.size()
    pos_emb=torch.tensor([[ math.cos(pos / 10000 ** ((2 * i +1)/ C)) if i % 2 == 1 else math.sin(pos / 10000 ** (2 * i / C)) for i in range(C)] for pos in range(T)])
    pos_emb.view(1,T,C)
    return pos_emb+x

pos_emb=Positional_encoding()
array=torch.tensor([[[2,2,3,4,6,1],[1,2,3,4,5,5],[1,2,3,4,5,5]],[[2,2,3,4,1,1],[1,2,3,4,2,2],[1,2,3,4,5,5]]],dtype=torch.float32)
pos_emb.forward(array)

tensor([[[2.0000, 3.0000, 3.0000, 5.0000, 6.0000, 2.0000],
         [1.8415, 2.9999, 3.0022, 5.0000, 5.0000, 6.0000],
         [1.9093, 2.9998, 3.0043, 5.0000, 5.0000, 6.0000]],

        [[2.0000, 3.0000, 3.0000, 5.0000, 1.0000, 2.0000],
         [1.8415, 2.9999, 3.0022, 5.0000, 2.0000, 3.0000],
         [1.9093, 2.9998, 3.0043, 5.0000, 5.0000, 6.0000]]])

In [70]:
class Self_attention(nn.Module):
  def __init__(self,head,emb_dim):
    super(Self_attention, self).__init__()
    self.dim=int(emb_dim/head)
    self.emb_dim=emb_dim
    self.head=head
    self.w_k=nn.Linear(emb_dim,emb_dim)
    self.w_q=nn.Linear(emb_dim,emb_dim)
    self.w_v=nn.Linear(emb_dim,emb_dim)
    self.w_o=nn.Linear(emb_dim,emb_dim)


  def forward(self,q=None,k=None,v=None,x=None,mask=None):
    if x!=None:
      B,T,C=x.size() #batch_size,sequence_size,encoding_dimension
      T=int(T)
      q,k,v=self.w_q(x),self.w_k(x),self.w_v(x)
      q,k,v=q.view(B,T,self.head,self.dim),k.view(B,T,self.head,self.dim),v.view(B,T,self.head,self.dim)
      q,k,v=(q.transpose(1,2)),k.transpose(1,2),v.transpose(1,2) #each head gets  seperate qkv dim -b head T embedding
      softmax=nn.Softmax(dim=-1)

    else:
      B,T,C=v.size()
      q,k,v=q,k,self.w_v(v)
      q,k,v=q.view(B,T,self.head,self.dim),k.view(B,T,self.head,self.dim),v.view(B,T,self.head,self.dim)
      q,k,v=(q.transpose(1,2)),k.transpose(1,2),v.transpose(1,2) #each head gets  seperate qkv dim -b head T embedding
      softmax=nn.Softmax(dim=-1)

    if mask==None:
      attention=softmax(torch.matmul(q,k.transpose(2,3)))
      attention=torch.matmul(attention,v)

    else:
      mask=torch.tril(torch.ones(self.dim,self.dim).view(1,1,self.dim,self.dim))
      attention=torch.matmul(q,k.transpose(2,3))
      attention=attention.masked_fill(mask[:,:,:,:]==0,-1e6)
      #for i in range(T):
        #for j in range(i+1,T):
          #attention[:,:,i,j]=-1e6
      attention=softmax(attention)
      attention=torch.matmul(attention,v)

    attention=attention.transpose(1,2).contiguous().view(B,T,C)
    attention=self.w_o(attention)
    return attention

array=torch.tensor([[[2,2,3,4,6,1],[1,2,3,4,5,5],[1,2,3,4,5,5]],[[2,2,3,4,1,1],[1,2,3,4,2,2],[1,2,3,4,5,5]]],dtype=torch.float32)
a=Self_attention(2,6)
a.forward(x=array,mask=True)

tensor([[[ 1.5500,  0.0532, -1.0138,  1.6964, -0.3638, -1.2509],
         [ 1.5474,  0.0544, -1.0094,  1.6921, -0.3564, -1.2314],
         [ 1.5449,  0.0564, -1.0060,  1.6894, -0.3516, -1.2174]],

        [[ 0.3530, -0.7131, -0.5057,  0.3594, -0.0983, -0.8502],
         [ 0.4449, -0.5823, -0.5251,  0.4445, -0.0979, -0.8111],
         [ 0.4189, -0.6340, -0.5240,  0.4428, -0.0979, -0.8345]]],
       grad_fn=<ViewBackward0>)

In [23]:
class Layer_norm(nn.Module):
  def __init__(self):
    super(Layer_norm,self).__init__()
    self.w=nn.parameter.Parameter(torch.ones(1))
    self.b=nn.parameter.Parameter(torch.zeros(1))

  def forward(self,x):
    mean=torch.mean(x)
    var=torch.var(x)
    x=(x-mean)/var
    x=self.w*x+self.b
    return x


layer=Layer_norm()
x=torch.tensor([[1,2,3,4,5]],dtype=torch.float32)
layer.forward(x)

tensor([[-0.8000, -0.4000,  0.0000,  0.4000,  0.8000]], grad_fn=<AddBackward0>)

In [22]:
class Encoder_block(nn.Module):
  def __init__(self,input_dim,h_dim,head):
    super(Encoder_block,self).__init__()
    self.in_dim=input_dim
    self.h_dim=h_dim
    self.head=head
    self.linear1=nn.Linear(self.in_dim,h_dim)
    self.linear2=nn.Linear(self.h_dim,self.in_dim)

  def forward(self,x):
    self_attention=Self_attention(self.head,self.in_dim)
    layer=Layer_norm()
    residual=x
    x=self_attention(x=x)
    x=layer(x+residual)
    layer2=Layer_norm()
    residual2=x
    x=F.gelu(self.linear1(x))
    x=self.linear2(x)
    x=layer2(x+residual2)
    return x

In [25]:
e=Encoder_block(6,4,2)
array=torch.tensor([[[2,2,3,4,6,1],[1,2,3,4,5,5],[1,2,3,4,5,5]],[[2,2,3,4,1,1],[1,2,3,4,2,2],[1,2,3,4,5,5]]],dtype=torch.float32)
e.forward(array)

tensor([[[-1.5685, -0.2285,  1.2707,  0.0991,  2.8894, -3.3499],
         [-2.3397, -0.0880,  1.2478,  0.3311,  1.9183,  1.0970],
         [-2.3397, -0.0880,  1.2478,  0.3311,  1.9183,  1.0970]],

        [[-1.4078,  0.5743,  0.8062,  0.4762, -1.6386, -2.5291],
         [-2.3696,  0.5848,  0.9089,  0.5564, -0.6804, -1.6592],
         [-2.6044,  0.4656,  0.8623,  0.6555,  2.2901,  1.2631]]],
       grad_fn=<AddBackward0>)

In [73]:
class Decoder_block(nn.Module):
  def __init__(self,input_dim,h_dim,head):
    super(Decoder_block,self).__init__()
    self.in_dim=input_dim
    self.head=head
    self.h_dim=h_dim
    self.linear1=nn.Linear(self.in_dim,h_dim)
    self.linear2=nn.Linear(self.h_dim,self.in_dim)

  def forward(self,q,k,x):
    masked_self_attention=Self_attention(self.head,self.in_dim)
    layer=Layer_norm()
    residual=x
    x=masked_self_attention(x=x,mask=True)
    x=layer(x+residual)


    layer2=Layer_norm()
    self_attention=Self_attention(self.head,self.in_dim)
    residual=x
    x=self_attention(q=q,k=k,v=x)
    x=layer2(x+residual)

    residual=x
    layer3=Layer_norm()
    x=F.gelu(self.linear1(x))
    x=self.linear2(x)
    x=layer3(x+residual)
    return x


In [209]:
e=Decoder_block(6,4,2)
array=torch.tensor([[[2,2,3,4,6,1],[1,2,3,4,5,5],[1,2,3,4,5,5]],[[2,2,3,4,1,1],[1,2,3,4,2,2],[1,2,3,4,5,5]]],dtype=torch.float32)
e.forward(array)

tensor([[[ 0.0371, -0.0079,  1.3871,  0.3024,  0.3389, -1.6315],
         [-0.1295, -0.1835,  1.1840,  0.3577,  0.0104, -0.1949],
         [-0.1291, -0.1887,  1.1821,  0.3527,  0.0105, -0.1986]],

        [[-0.0432, -0.1530,  1.1782,  0.3108, -1.6232, -1.7156],
         [-0.4390, -0.1456,  1.1547,  0.3261, -1.1648, -1.3221],
         [-0.3177, -0.1530,  0.9937,  0.4494,  0.2062, -0.0410]]],
       grad_fn=<AddBackward0>)

In [74]:
class Transformer(nn.Module):
  def __init__(self,input_dim,n,head,h_dim):
    super(Transformer,self).__init__()
    self.n=n
    self.h_dim=h_dim
    self.head=head
    self.in_dim=input_dim
    self.encoder_list=[ Encoder_block(self.in_dim,self.h_dim,self.head) for i in range(n)]
    self.decoder_list=[ Decoder_block(self.in_dim,self.h_dim,self.head) for i in range(n)]
    self.w_q,self.w_k=nn.Linear(self.in_dim,self.in_dim),nn.Linear(self.in_dim,self.in_dim)
    self.linear1=nn.Linear(self.in_dim,h_dim)
    self.linear2=nn.Linear(self.h_dim,self.in_dim)

  def forward(self,x,y):
    pos1,pos2=Positional_encoding(),Positional_encoding()
    x,y=pos1(x),pos2(y)
    for encoder in self.encoder_list:
      x=encoder(x)
    q,k=self.w_q(x),self.w_k(x)
    x=y
    for decoder in self.decoder_list:
      x=decoder(q,k,x)
    x=F.gelu(self.linear1(x))
    x=F.softmax(self.linear2(x),dim=-1)
    return x

transformer=Transformer(6,2,2,24)
array=torch.tensor([[[2,2,3,4,6,1],[1,2,3,4,5,5],[1,2,3,4,5,5]],[[2,2,3,4,1,1],[1,2,3,4,2,2],[1,2,3,4,5,5]]],dtype=torch.float32)
transformer.forward(array,array)

tensor([[[0.1129, 0.1441, 0.2013, 0.2410, 0.1234, 0.1774],
         [0.1010, 0.1104, 0.2203, 0.2697, 0.0649, 0.2337],
         [0.0977, 0.1080, 0.2235, 0.2767, 0.0613, 0.2328]],

        [[0.0844, 0.1064, 0.2257, 0.3596, 0.0799, 0.1440],
         [0.0924, 0.1046, 0.2186, 0.3480, 0.0799, 0.1564],
         [0.0939, 0.1064, 0.2274, 0.2645, 0.0587, 0.2490]]],
       grad_fn=<SoftmaxBackward0>)

In [ ]:
()